## Practice 5-1

In [3]:
from dotenv import load_dotenv
load_dotenv()

import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [18]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# Tool 정의 
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [16]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하는 함수 
def search_wiki(input_data: dict) -> List[Document]:
    """Search Wikipedia documents based on user input (query) and return k documents"""
    query = input_data["query"]
    k = input_data.get("k", 2)  
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSearchSchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documents to return (default is 2)")

# RunnableLambda 함수를 사용하여 위키피디아 문서 로더를 Runnable로 변환 
runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema
)
print(type(wiki_search))

# 위키 검색 실행
query = "커피의 유래"
wiki_results = wiki_search.invoke({"query":query})

# 검색 결과 출력
for result in wiki_results:
    print(type(result))
    print(result)  
    print("-" * 100)  

<class 'langchain_core.tools.structured.StructuredTool'>
<class 'langchain_core.documents.base.Document'>
page_content='커피(영어: coffee, 咖啡, 珈琲, 가배, 문화어: 흑차, 검은차), 커피차 또는 커피음료(영어: coffee beverage)는 커피 나무 열매의 씨(커피콩)를 볶아 가루로 낸 것을 따뜻한 물과 차가운 물 또는 증기로 우려내어 마시는, 쓴맛이 나는 짙은 갈색의 음료이다.
색이 어둡고 쓴맛이 있으며 약산성인 커피는 주로 카페인 함량으로 인해 인간에게 자극 효과를 준다. 전세계 뜨거운 음료 시장에서 가장 높은 판매량을 기록하고 있다.
커피나무 열매의 씨앗을 분리하여 볶지 않은 녹색 커피콩을 생산한다. 원두를 로스팅한 다음 일반적으로 뜨거운 물에 담근 후 미세한 입자로 갈아서 걸러내어 커피 한 잔을 만든다. 일반적으로 뜨겁게 제공되지만 차가운 아이스 커피가 일반적이다. 커피는 다양한 방법으로 준비되고 제공될 수 있다(예: 에스프레소, 프렌치 프레스, 카페 라떼 또는 이미 끓인 캔커피). 쓴맛을 가리거나 풍미를 강화하기 위해 설탕, 설탕 대체물, 우유, 크림을 첨가하는 경우가 많다.
커피는 이제 세계적인 상품이 되었지만 홍해 주변의 음식 전통과 밀접하게 연관되어 있는 오랜 역사를 가지고 있다. 현대 음료로서 커피를 마시는 것에 대한 가장 신뢰할 수 있는 증거는 15세기 중반 아라비아 남부의 현대 예멘 수피 사원에서 나타난다. 그곳에서 커피 씨앗은 현재 준비되는 방식과 유사한 방식으로 처음으로 로스팅되고 양조되었다. 커피 원두는 소말리아 해안 중개인을 통해 에티오피아고원의 예멘인이 조달하여 예멘에서 재배했다. 16세기에 이 음료는 중동과 북아프리카의 나머지 지역으로 퍼져나갔고 나중에 유럽으로 퍼졌다.
가장 일반적으로 재배되는 두 가지 커피 콩 유형은 C. 아라비카와 C. 로부스타이다. 커피나무는 70여 개국, 주로 아메리카 대륙, 동남아시아, 인도 아

In [22]:
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_search])

query = "서울 인사동의 유명 카페는 어디야? 한국 내 커피의 유래도 알려줘"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PqhatrxtB7Erz9j9IclLDlG1', 'function': {'arguments': '{"query": "서울 인사동의 유명 카페"}', 'name': 'tavily_search_func'}, 'type': 'function'}, {'id': 'call_tQ1X0QSijFBxnXAf0XZfkpn6', 'function': {'arguments': '{"query": "한국 내 커피의 유래"}', 'name': 'wiki_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 196, 'total_tokens': 268, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bj4NoYm8yhncHpbNsrUWorf4I6Opv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cf47556a-1afd-454c-8e96-fcaf10c8ad68-0', tool_calls=[{'name': 'tavily_search_func', 'args': {'query': '서울 인사동의 유명 카페'}, 'id': 'call

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"커의 유래"})
pprint(summarized_text)

class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_summary])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 인사동의 인기있는 카페를 알려줘. "
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg.content)
print("-" * 100)


('**유월절 (Passover)**: 유월절은 유대인들이 이집트에서의 노예 생활에서 탈출한 사건을 기념하는 절기로, 유대력 니산월 14일 '
 "저녁에 시작된다. 이 절기는 '장자들의 죽음'에서 넘어갔다는 의미를 지니며, 유대교에서는 봄의 축제인 '하그 에 아비브'와 자유의 때를 "
 "강조하는 '즈만 헤루테누'로 불린다. 구약 성경에서는 흠 없는 어린 양을 잡아 그 피를 문에 바르고 무교병과 쓴 나물을 함께 먹는 의식이 "
 '포함되어 있다. 신약 성경에서는 예수의 최후의 만찬과 관련이 있다. 현대 유대교에서는 여전히 이 절기를 지키고 있으며, 기독교에서는 '
 '예수의 죽음을 기념하는 방식으로 기념된다.\n'
 '\n'
 '**홍콩 (Hong Kong)**: 홍콩은 중화인민공화국의 특별행정구로, 주강 삼각주에 위치하며, 세계에서 가장 인구 밀도가 높은 지역 '
 '중 하나이다. 1842년 영국에 양도된 후 중요한 금융 센터로 발전하였고, 1997년 중국에 반환되었다. 홍콩은 높은 GDP와 소득 '
 '불평등을 겪고 있으며, 잘 발달된 대중교통 시스템을 갖추고 있다. 홍콩의 이름은 "향기로운 항구"라는 의미를 지니며, 역사적으로 다양한 '
 '문화와 상업적 교류의 중심지로 자리잡았다.')
''
----------------------------------------------------------------------------------------------------


In [27]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_summary])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def wiki_summary_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = wiki_summary.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

response = wiki_summary_chain.invoke("커피의 유래에 대해서 알려주세요.")

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_lXCo4faJAN3n3aAGrhnZJ9Xp', 'function': {'arguments': '{"query":"커피의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 120, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bj4S0hCLRR3EzQtPmqNgvhXDxXvk1', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--001ad38a-8289-4db2-92df-d5753e46e773-0' tool_calls=[{'name': 'wiki_summary', 'args': {'query': '커피의 유래'}, 'id': 'call_lXCo4faJAN3n3aAGrhnZJ9Xp', 'type': 'tool_call'}] usage_metadata={'input_tokens': 120, 'output_tokens': 19, 'total_tokens': 139, 'input_token_details': {'a

In [8]:
from langchain.document_loaders import TextLoader
from langchain_core.documents import Document

loader = TextLoader("./data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents

menu_documents = [] # [Document, Document]
for doc in documents:
    menu_documents += split_menu_items(doc)
    
    
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(type(doc))
    pprint(vars(doc))
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.
<class 'langchain_core.documents.base.Document'>
{'id': None,
 'metadata': {'menu_name': '아메리카노',
              'menu_number': 1,
              'source': './data/cafe_menu_data.txt'},
 'page_content': '1. 아메리카노\n'
                 '   • 가격: ₩4,500\n'
                 '   • 주요 원료: 에스프레소, 뜨거운 물\n'
                 '   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 '
                 '잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.',
 'type': 'Document'}

메뉴 번호: 1
메뉴 이름: 아메리카노
내용:
1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 ...
<class 'langchain_core.documents.base.Document'>
{'id': None,
 'metadata': {'menu_name': '카페라떼',
              'menu_number': 2,
              'source': './data/cafe_menu_data.txt'},
 'page_content': '2. 카페라떼\n'
                 '   • 가격: ₩5,500\n'
                 '   • 주요 원료: 에스프레소, 스팀 밀크\n'
                 '   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 '
      

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

# FAISS 인덱스 생성
cafe_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
cafe_db.save_local("./db/cafe_db")


# Retriever 생성
cafe_retriever = cafe_db.as_retriever(
    search_kwargs={'k': 4},
)



검색 결과: 4개
메뉴 번호: 10
메뉴 이름: 티라미수

메뉴 번호: 5
메뉴 이름: 카라멜 마키아토

메뉴 번호: 4
메뉴 이름: 바닐라 라떼

메뉴 번호: 8
메뉴 이름: 녹차 라떼



In [29]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# menu db 벡터 저장소 로드
cafe_db = FAISS.load_local(
    "./db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = cafe_db.similarity_search(query, k=4)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

print("name: ")
print(db_search_cafe_func.name)

name: 
db_search_cafe_func


In [32]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def cafe_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        pprint(f"{tool_call['name']}: \n{tool_call}")
        print("%"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

    print("tool_msgs: \n") 
    for tool in tool_msgs:
        pprint(tool.name)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

response = cafe_menu_chain.invoke("아메리카노의 가격과 특징은 무엇인가요?")

# 응답 출력 
print(response.content)


('db_search_cafe_func: \n'
 "{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': "
 "'call_BeKkDEK6kUGr9XB7KRpxRdat', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
('wiki_summary: \n'
 "{'name': 'wiki_summary', 'args': {'query': '아메리카노'}, 'id': "
 "'call_3YNhMRCdNtLQn78JVAkxffNY', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
tool_msgs: 

'db_search_cafe_func'
'wiki_summary'
----------------------------------------------------------------------------------------------------
### 아메리카노의 가격
아메리카노의 가격은 보통 **4,500원**입니다. 카페에 따라 가격이 조금씩 상이할 수 있습니다.

### 아메리카노의 특징
아메리카노는 에스프레소에 뜨거운 물을 추가하여 만든 커피 음료입니다. 이는 에스프레소의 진한 맛을 물로 희석하여 부드럽고 깔끔한 맛을 즐길 수 있게 해줍니다. 아메리카노는 원두의 품질과 추출 방법에 따라 다양한 풍미를 나타낼 수 있으며, 일반적으로 다음과 같은 특성을 가지고 있습니다:

- **풍미**: 진한 커피향과 부드러운 맛
- **커피와 물의 비율**: 에스프레소 1샷에 대해 물이 보통 2배 이상 추가됩니다.
- **개성과 다양성**: 아메리카노는 다른 첨가물을 추

## Practice 5-2

In [35]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("아메리카노 정보와 커피 역사를 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_cafe_func", "args": {"query": "아메리카노"}, "id": "1"}]),
    ToolMessage("아메리카노 정보 반환", tool_call_id="1"),    
    AIMessage("아메리카노의 가격은 4,500원이며 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "아메리카노", "k": 1}, "id": "2"}]),
    ToolMessage("커피 역사의 정보 반환", tool_call_id="2"),
    AIMessage("종합된 최종 답변", name="example_assistant")
]

system = """당신은 카페 메뉴 정보와 일반적인 음식/음료 지식을 제공하는 AI입니다.

도구 사용 가이드라인:
- db_search_cafe_func: 카페 메뉴 정보 (가격, 재료, 설명)
- wiki_summary: 일반 지식 (역사, 제조법, 문화적 배경)  
- tavily_search_func: 최신 정보 (트렌드, 뉴스, 실시간 정보)

사용 원칙:
1. 카페 메뉴 관련 질문 → 반드시 메뉴 DB 먼저 검색
2. 역사/문화/일반 지식 → 위키피디아 활용
3. 최신 트렌드/뉴스 → 웹 검색 활용
4. 복합 질문 → 여러 도구 순차 사용
5. 정보 출처를 명확히 구분하여 답변
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

In [41]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
system = """"당신은 카페 메뉴 정보와 일반적인 음식/음료 지식을 제공하는 AI입니다.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

In [40]:
@chain
def cafe_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = fewshot_search_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)
        
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
            
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

In [42]:
# 체인 실행
query = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요"
response = cafe_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

db_search_cafe_func: 
{'name': 'db_search_cafe_func', 'args': {'query': '카페라떼 디저트 추천'}, 'id': 'call_pGp9BMLaKCeR3MFzEMEm3fp0', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '카페라떼'}, 'id': 'call_Aj34SBjyHBm7CHVuZa3uaHnl', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='59257a6b-d6d5-4e44-9b07-3ed5439bfcfc', metadata={'source': './data/cafe_menu_data.txt', 'menu_number': 4, 'menu_name': '바닐라 라떼'}, page_content='4. 바닐라 라떼\\n   • 가격: ₩6,000\\n   • 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽\\n   • 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.'), Document(id='f44017e3-b32e-4eda-a838-b0565978cbba', metadata={'source': './data/cafe_menu_data.txt', 'menu_number': 2, 'menu_name': '카페라떼'}, page_content='2. 카